In [189]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn import linear_model
from IPython.display import Image

plt.rcParams['figure.figsize'] = (20, 12)

## Dataset: 100 Portfolios by Kenneth French  ##

In [197]:
# 100 portfolios excess returns on 100 portfolios sorted on size and book-to-market
df = pd.read_csv(r"100_Portfolios_10x10.csv")
df.rename(columns = {"Unnamed: 0": "Time"}, inplace=True)
df

C:\Users\shena\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Time,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,ME1 BM5,ME1 BM6,ME1 BM7,ME1 BM8,ME1 BM9,...,BIG LoBM,ME10 BM2,ME10 BM3,ME10 BM4,ME10 BM5,ME10 BM6,ME10 BM7,ME10 BM8,ME10 BM9,BIG HiBM
0,192607,-99.99,12.3656,-99.99,-20.6349,-6.4864,-4.7429,-4.8812,15.7276,-1.6398,...,3.3411,1.182,19.0161,1.4227,1.9459,2.4834,3.1644,0.3378,0.8576,-99.99
1,192608,-99.99,2.9904,-99.99,8,6.7495,-3.5639,-4.1188,-1.6529,-5.3824,...,0.2379,3.9628,8.5675,3.0099,1.1427,3.0263,6.0936,6.0606,11.2245,-99.99
2,192609,-99.99,-18.583,-99.99,-3.7037,-5.0512,-2.1739,-4.5714,-10.61,0.4919,...,-0.392,-1.9836,10.7849,0.8073,-0.9185,1.0889,-0.5436,-0.9524,-1.0703,-99.99
3,192610,-99.99,-4.1369,-99.99,-11.5385,-5.3996,-17.7778,3.5554,-9.2233,-9.1593,...,-4.32,-0.648,-6.6205,-0.5337,-1.5745,-1.9202,-5.3535,-0.9615,-3.9246,-99.99
4,192611,-99.99,-8.2589,-99.99,34.7826,1.2296,0.2027,-4.8789,-3.201,-1.8459,...,6.9978,-0.0541,0.4585,2.9192,1.1447,0.8811,3.6576,3.7217,3.268,-99.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8393,202007,0.1071,0.0467,0.2336,0.2217,0.0754,0.1764,0.0633,0.0592,0.0488,...,0.1155,0.0488,0.1478,0.2332,0.2356,0.1679,0.1511,0.0778,0.0392,0.0235
8394,202008,0.0955,0.007,0.2215,0.2197,0.0526,0.1665,0.0589,0.0365,0.0446,...,0.1171,0.0505,0.1491,0.2338,0.2299,0.1734,0.1656,0.0802,0.0395,0.0225
8395,202009,0.1291,0.0475,0.2427,0.2223,0.0864,0.1716,0.063,0.0306,0.0454,...,0.1155,0.0519,0.1532,0.2393,0.2254,0.1692,0.1689,0.0822,0.0397,0.0225
8396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [201]:
# Convert first column to datatime format
df_date = df.copy()
df_date = df_date.dropna()

for date in df_date['Time']:
    date = str(date)+"01"
modified_dates = [str(date) + "01" for date in df_date['Time']]
df_date['Time'] = modified_dates

df_date['Time_length'] = df_date['Time'].map(len)
df_date = df_date[df_date['Time_length'] == 8]

In [203]:
df_date['Time'] = pd.to_datetime(df_date['Time'])
df_date

,Time,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,ME1 BM5,ME1 BM6,ME1 BM7,ME1 BM8,ME1 BM9,...,ME10 BM2,ME10 BM3,ME10 BM4,ME10 BM5,ME10 BM6,ME10 BM7,ME10 BM8,ME10 BM9,BIG HiBM,Time_length
0,1926-07-01,-99.99,12.3656,-99.99,-20.6349,-6.4864,-4.7429,-4.8812,15.7276,-1.6398,...,1.182,19.0161,1.4227,1.9459,2.4834,3.1644,0.3378,0.8576,-99.99,8
1,1926-08-01,-99.99,2.9904,-99.99,8,6.7495,-3.5639,-4.1188,-1.6529,-5.3824,...,3.9628,8.5675,3.0099,1.1427,3.0263,6.0936,6.0606,11.2245,-99.99,8
2,1926-09-01,-99.99,-18.583,-99.99,-3.7037,-5.0512,-2.1739,-4.5714,-10.61,0.4919,...,-1.9836,10.7849,0.8073,-0.9185,1.0889,-0.5436,-0.9524,-1.0703,-99.99,8
3,1926-10-01,-99.99,-4.1369,-99.99,-11.5385,-5.3996,-17.7778,3.5554,-9.2233,-9.1593,...,-0.648,-6.6205,-0.5337,-1.5745,-1.9202,-5.3535,-0.9615,-3.9246,-99.99,8
4,1926-11-01,-99.99,-8.2589,-99.99,34.7826,1.2296,0.2027,-4.8789,-3.201,-1.8459,...,-0.0541,0.4585,2.9192,1.1447,0.8811,3.6576,3.7217,3.268,-99.99,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8391,2020-05-01,0.1978,0.1872,0.1159,0.2837,0.1398,0.3236,0.1116,0.069,0.1022,...,0.0062,0.0681,0.0736,0.0399,0.2118,0.0441,0.1296,0.0163,0.0223,8
8392,2020-06-01,0.1457,0.1924,0.1168,0.2992,0.0904,0.3665,0.1075,0.0726,0.09,...,0.0054,0.0694,0.0722,0.0418,0.2056,0.0469,0.1302,0.0159,0.0215,8
8393,2020-07-01,0.1071,0.0467,0.2336,0.2217,0.0754,0.1764,0.0633,0.0592,0.0488,...,0.0488,0.1478,0.2332,0.2356,0.1679,0.1511,0.0778,0.0392,0.0235,8
8394,2020-08-01,0.0955,0.007,0.2215,0.2197,0.0526,0.1665,0.0589,0.0365,0.0446,...,0.0505,0.1491,0.2338,0.2299,0.1734,0.1656,0.0802,0.0395,0.0225,8


In [188]:
# Reformat titles
df[0] = pd.to_datetime(df[0])
df = df.rename(columns={0:'Date'})
df = df.set_index('Date')
df = df.astype('float64')

KeyError: 0

In [ ]:
# difference the non-stationary series to stationarity, and standardize
# the series to have zero mean and unit variance before extracting principal components